I wrote SQL queries and ran them on sqlliteonline.com. This is all that code documented, just for reference.

In [ ]:
SELECT DISTINCT "ASIN"
FROM amazon_data;  -- to check how many unique products there are

In [ ]:
SELECT "Brand", COUNT(*) AS count
FROM amazon_data
GROUP BY "Brand"
ORDER BY count DESC;  -- to see brands, and to check if they are actual brands, or some error in webscraping

In [ ]:
-- STANDARDIZING BRAND NAMES
UPDATE amazon_data
SET "Brand" = CASE
	WHEN LOWER("Brand") = 'boat' THEN 'boAt'
    WHEN LOWER("Brand") = 'oneplus' THEN 'OnePlus'
    WHEN LOWER("Brand") = 'amazon' THEN 'Amazon Basics'
    WHEN "Brand" IN ('Q', 'W20', 'Mustang') then 'Boult'
    ELSE "Brand"
    END;

In [ ]:
-- CREATE A NEW TABLE FOR DEDUPLICATION
CREATE TABLE cleaned_amazon_data AS
SELECT
    "Title",
    "Brand",
    "Price",
    "MRP",
    "Discount",
    "Rating",
    "Review_Count",
    "Prime",
    "ASIN",
    "URL"
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY "ASIN" ORDER BY "Title") as rn
    FROM amazon_data
    WHERE "ASIN" IS NOT NULL AND "ASIN" != ''
) ranked
WHERE rn = 1;

In [ ]:
-- CLEAN PRICE COLUMN
UPDATE cleaned_amazon_data
SET "Price" = REGEXP_REPLACE("Price", '[^0-9]', '', 'g')::INTEGER
WHERE "Price" IS NOT NULL AND "Price" ~ '[0-9]';

-- CLEAN MRP COLUMN
UPDATE cleaned_amazon_data
SET "MRP" = REGEXP_REPLACE("MRP", '[^0-9]', '', 'g')::INTEGER
WHERE "MRP" IS NOT NULL AND "MRP" ~ '[0-9]';

-- CLEAN DISCOUNT COLUMN
UPDATE cleaned_amazon_data
SET "Discount" = SUBSTRING("Discount" FROM '\((\d+)%')::INTEGER
WHERE "Discount" IS NOT NULL AND "Discount" ~ '\(\d+%';

-- CLEAN RATING COLUMN
UPDATE cleaned_amazon_data
SET "Rating" = SPLIT_PART("Rating", ' ', 1)::FLOAT
WHERE "Rating" IS NOT NULL AND "Rating" ~ '^[0-9\.]+';

-- CLEAN REVIEW_COUNT COLUMN
UPDATE cleaned_amazon_data
SET "Review_Count" = REPLACE("Review_Count", ',', '')::INTEGER
WHERE "Review_Count" IS NOT NULL AND "Review_Count" ~ '^[0-9,]+$';

-- CLEAN PRIME COLUMNN
UPDATE cleaned_amazon_data
SET "Prime" = CASE WHEN LOWER("Prime") = 'true' THEN 'TRUE' ELSE 'FALSE' END;

-- CHECK THE TABLE AFTER CLEANING
SELECT * FROM cleaned_amazon_data LIMIT 5;

In [ ]:
-- CHECK DATA TYPES OF ALL COLUMNS, AND CONVERT NUMERIC ONES TO INTEGERS
SELECT
  column_name,
  data_type
FROM information_schema.columns
WHERE table_name = 'cleaned_amazon_data';

-- Need to convert '' to NULL first, SQL can't convert empty strings to INTEGER or whatever
-- Clean Price
UPDATE cleaned_amazon_data
SET "Price" = NULL
WHERE "Price" = '';

ALTER TABLE cleaned_amazon_data
ALTER COLUMN "Price" TYPE INTEGER USING "Price"::INTEGER;

-- Clean MRP
UPDATE cleaned_amazon_data
SET "MRP" = NULL
WHERE "MRP" = '';

ALTER TABLE cleaned_amazon_data
ALTER COLUMN "MRP" TYPE INTEGER USING "MRP"::INTEGER;

-- Clean Review_Count
UPDATE cleaned_amazon_data
SET "Review_Count" = NULL
WHERE "Review_Count" = '';

ALTER TABLE cleaned_amazon_data
ALTER COLUMN "Review_Count" TYPE INTEGER USING "Review_Count"::INTEGER;

-- Clean Discount
UPDATE cleaned_amazon_data
SET "Discount" = NULL
WHERE "Discount" = '';

ALTER TABLE cleaned_amazon_data
ALTER COLUMN "Discount" TYPE INTEGER USING "Discount"::INTEGER;

-- Clean Rating
UPDATE cleaned_amazon_data
SET "Rating" = NULL
WHERE "Rating" = '';

ALTER TABLE cleaned_amazon_data
ALTER COLUMN "Rating" TYPE FLOAT USING "Rating"::FLOAT;


In [ ]:
-- CREATE SAVINGS COLUMN
ALTER TABLE cleaned_amazon_data ADD COLUMN "Savings" INTEGER;

UPDATE cleaned_amazon_data
SET "Savings" = "MRP" - "Price"
WHERE "MRP" IS NOT NULL AND "Price" IS NOT NULL;

In [ ]:
-- DROP DELIVERY INFO AND AVAILABILITY
ALTER TABLE cleaned_amazon_data DROP COLUMN "Delivery_Details";
ALTER TABLE cleaned_amazon_data DROP COLUMN "Availability";


In [ ]:
-- DROP ROWS WITH MISSING PRICE
DELETE FROM cleaned_amazon_data
WHERE "Price" IS NULL OR "Price" = '' OR "Price" = '₹';


In [ ]:
-- CHECK AFTER BASIC CLEANING
-- Check remaining data quality
SELECT
    COUNT(*) as total_rows,
    COUNT("Price") as rows_with_price,
    COUNT("MRP") as rows_with_mrp,
    COUNT("Rating") as rows_with_rating
FROM cleaned_amazon_data;

In [ ]:
-- CHECK FOR BROKEN URLs
SELECT "Title", "URL"
FROM cleaned_amazon_data
WHERE
  "URL" ILIKE '%javascript%'
  OR "URL" ILIKE '%void%'
  OR "URL" ILIKE '%0)%';

-- FIX BROKEN URLs BY RECONSTRUCTING THEM WITH ASIN
UPDATE cleaned_amazon_data
SET "URL" = 'https://www.amazon.in/dp/' || "ASIN"
WHERE "URL" ILIKE '%javascript:void(0)%';